In [39]:
# Import base dependencies
import pandas as pd
import requests
import time
from datetime import datetime, timedelta
import math
import pyodbc
from config import av_api_key as api_key, seeking_alpha_name, seeking_alpha_pw

# Import dependencies for web scraping
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import numpy as np
import os

# import matplotlib.pyplot as plt
# from sklearn.linear_model import LinearRegression

# Import ML dependencies
# import tensorflow as tf
# from tensorflow import keras
# from keras.models import Sequential
# from keras.layers import SimpleRNN, Flatten, TimeDistributed, LSTM

In [40]:
def load_access_table_to_df(access_file_path, table_name):
    """
    Loads a table from a Microsoft Access database into a Pandas DataFrame.

    Parameters:
    access_file_path (str): The full path to the Microsoft Access database file (.mdb or .accdb).
    table_name (str): The name of the table to load from the database.

    Returns:
    pd.DataFrame: The DataFrame containing the data from the specified table.

    Notes:
    - Requires the pyodbc library and the Microsoft Access Driver installed on your system.
    - Ensure the bit version (32-bit or 64-bit) of Python matches the installed Access Driver.
    """
    # Connection string for Microsoft Access
    conn_str = (
        r"Driver={Microsoft Access Driver (*.mdb, *.accdb)};"
        r"DBQ=" + access_file_path + ";"
    )
    
    # Establish connection
    conn = pyodbc.connect(conn_str)
    
    # SQL query to select all data from the table
    query = f"SELECT * FROM {table_name}"
    
    # Read data into DataFrame
    df = pd.read_sql(query, conn)
    
    # Close the connection
    conn.close()
    
    return df

def create_db_dfs(table_name='brokerage'):
    # Path to database
    access_file_path = 'c:/users/culle/onedrive/documents/finances/investments.accdb'

    # Individual tables in database
    table_name = table_name
    
    return load_access_table_to_df(access_file_path,table_name)

In [41]:
def create_summary_dataframe_with_weighted_date(df, reference_date=None):
    """
    Creates a summary DataFrame by aggregating transaction data for each company (TICKER),
    including a weighted average purchase date based on total cost (quantity * purchase price).

    The summary includes:
    - TICKER: The company ticker.
    - ASSET_CLASS: The asset class (assumed consistent per TICKER, takes the first occurrence).
    - SECTOR: The sector (assumed consistent per TICKER, takes the first occurrence).
    - WEIGHTED_AVG_PURCHASE_DATE: The cost-weighted average purchase date.
    - TOTAL_COST: The total cost of all purchases.
    - TOTAL_QUANTITY: The total quantity of shares purchased.
    - AVG_PURCHASE_PRICE: The weighted average purchase price (total cost / total quantity).

    Parameters:
    df (pd.DataFrame): The input DataFrame with columns: ID, TICKER, ASSET_CLASS, SECTOR,
                      ACQUIRED, PURCHASE_PRICE, QUANTITY.
    reference_date (str): Reference date for date-to-numeric conversion (default: '2025-01-01').

    Returns:
    pd.DataFrame: The summary DataFrame with aggregated data per TICKER, including weighted purchase date.
    """
    # Set reference_date to current date if None
    if reference_date is None:
        reference_date = datetime.now().date()

    # Ensure ACQUIRED is in datetime format
    df['ACQUIRED'] = pd.to_datetime(df['ACQUIRED'])
    
    # Calculate the cost for each transaction
    df['COST'] = df['PURCHASE_PRICE'] * df['QUANTITY']
    
    # Convert ACQUIRED date to numeric (days since reference_date)
    reference_date = pd.to_datetime(reference_date)
    df['DAYS_SINCE_REF'] = (df['ACQUIRED'] - reference_date).dt.days
    
    # Calculate weighted days (cost * days since reference)
    df['WEIGHTED_DAYS'] = df['COST'] * df['DAYS_SINCE_REF']
    
    # Group by TICKER and aggregate
    summary_df = df.groupby('TICKER').agg({
        'ASSET_CLASS': 'first',
        'ALPHA_PICKED': 'first',
        'QUANT_RATING': 'first',
        'SECTOR': 'first',
        'ACQUIRED': 'min',
        'QUANTITY': 'sum',
        'COST': 'sum',
        'WEIGHTED_DAYS': 'sum'
    }).reset_index()
    
    # Rename columns for clarity
    summary_df = summary_df.rename(columns={
        'ACQUIRED': 'FIRST_ACQUIRED',
        'QUANTITY': 'TOTAL_QUANTITY',
        'COST': 'TOTAL_COST'
    })
    
    # Calculate weighted average purchase price
    summary_df['AVG_PURCHASE_PRICE'] = round(summary_df['TOTAL_COST'] / summary_df['TOTAL_QUANTITY'], ndigits=3)
    
    # Calculate weighted average days
    summary_df['WEIGHTED_AVG_DAYS'] = summary_df['WEIGHTED_DAYS'] / summary_df['TOTAL_COST']
    
    # Convert weighted average days back to a datetime
    summary_df['WEIGHTED_AVG_PURCHASE_DATE'] = reference_date + pd.to_timedelta(summary_df['WEIGHTED_AVG_DAYS'], unit='D')
    
    # Round the weighted average date to the nearest day for clarity
    summary_df['WEIGHTED_AVG_PURCHASE_DATE'] = summary_df['WEIGHTED_AVG_PURCHASE_DATE'].dt.round('D')
    
    # Reorder columns as requested
    columns_order = [
        'TICKER', 'ALPHA_PICKED', 'QUANT_RATING', 'ASSET_CLASS', 'SECTOR', 'WEIGHTED_AVG_PURCHASE_DATE',
        'TOTAL_COST', 'TOTAL_QUANTITY', 'AVG_PURCHASE_PRICE'
    ]
    summary_df = summary_df[columns_order]
    
    # Drop temporary columns from input DataFrame
    df.drop(['COST', 'DAYS_SINCE_REF', 'WEIGHTED_DAYS'], axis=1, inplace=True)
    
    return summary_df

In [42]:
# Load database tables
r_analysis = create_db_dfs('retirement')
alpha_raw = create_db_dfs('alpha_picks')
watch_list = create_db_dfs('watch_list')
b_analysis = create_db_dfs('brokerage')
york_401k_list = create_db_dfs('york_401k')

C:\Users\culle\AppData\Local\Temp\ipykernel_28420\1991132214.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [43]:
# Merge portfolios and watch lists to create master portfolio
r_analysis2 = pd.merge(
        r_analysis,
        alpha_raw, 
        on='TICKER',
        how='outer'
    )
r_analysis2.drop(columns=['ID_x','ID_y'], inplace=True)

alpha_plus_watch = pd.concat([alpha_raw, watch_list], ignore_index=True)

b_analysis2 = pd.merge(
    b_analysis,
    alpha_plus_watch,
    on='TICKER',
    how='outer'
)
b_analysis2.drop(columns=['ID_x','ID_y'], inplace=True)

In [44]:
# Create summary tables of portfolios
brokerage_df = create_summary_dataframe_with_weighted_date(b_analysis2)
ira_df = create_summary_dataframe_with_weighted_date(r_analysis2)

In [45]:
def get_history(symbol, api_key, days=252):  # ~1 year default
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&apikey={api_key}&outputsize=full&entitlement=delayed"
    try:
        response = requests.get(url).json()
        if "Time Series (Daily)" not in response:
            error_msg = response.get('Note', response.get('Information', 'Unknown error'))
            print(f"Error fetching price data for {symbol}: {error_msg}")
            print(f"Response keys: {list(response.keys())}")
            return None
        
        time_series = response["Time Series (Daily)"]
        df = pd.DataFrame.from_dict(time_series, orient="index", dtype=float)
        
        # Debug: Print available columns
        # print(f"Columns for {symbol}: {list(df.columns)}")
        
        # Rename columns dynamically
        column_map = {
            col: name for col, name in [
                ("1. open", "Open"), ("2. high", "High"), ("3. low", "Low"),
                ("4. close", "Close"), ("5. volume", "Volume"), ("6. volume", "Volume"),
                ("7. adjusted close", "Adjusted Close"), ("8. dividend amount", "Dividend")
            ] if col in df.columns
        }
        if "5. volume" not in df.columns and "6. volume" not in df.columns:
            print(f"No volume data for {symbol}")
            return None
        
        df = df.rename(columns=column_map)
        df.index = pd.to_datetime(df.index)
        df = df.sort_index().tail(days)
        return df
    except Exception as e:
        print(f"Exception fetching price data for {symbol}: {str(e)}")
        return None

def get_fundamentals(symbol, api_key, current_price):
    url = f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={api_key}&entitlement=delayed"
    try:
        response = requests.get(url).json()
        if not response or "Symbol" not in response:
            error_msg = response.get('Note', response.get('Information', 'No data'))
            print(f"Error fetching fundamentals for {symbol}: {error_msg}")
            print(f"Full response: {response}")
            return None
        
        def safe_float(value, default):
            if value in [None, 'None', '']:
                return default
            try:
                return float(value)
            except (ValueError, TypeError):
                return default
        
        pe_ratio = safe_float(response.get('PERatio'), float('inf'))
        pb_ratio = safe_float(response.get('PriceToBookRatio'), float('inf'))
        
        # Calculate EPS and Book Value
        eps = current_price / pe_ratio if pe_ratio != float('inf') and pe_ratio != 0 else 0
        book_value = current_price / pb_ratio if pb_ratio != float('inf') and pb_ratio != 0 else 0
        
        fundamentals = {
            'pe_ratio': pe_ratio,
            'pb_ratio': pb_ratio,
            'dividend_yield': safe_float(response.get('DividendYield'), 0),
            'debt_to_equity': safe_float(response.get('DebtToEquityRatio'), float('inf')),
            'eps': eps,
            'book_value': book_value
        }
        
        return fundamentals
    except Exception as e:
        print(f"Exception fetching fundamentals for {symbol}: {str(e)}")
        return None

def calculate_vwap(df, days=126):  # ~2 quarters
    if 'Volume' not in df.columns:
        print("Missing Volume column in DataFrame")
        return None
    
    vwap_analysis = df[-days:].copy()
    vwap_analysis['Cumulative_LTPV'] = (vwap_analysis['Low'] * vwap_analysis['Volume']).cumsum()
    vwap_analysis['Cumulative_HTPV'] = (vwap_analysis['High'] * vwap_analysis['Volume']).cumsum()
    vwap_analysis['Cumulative_Volume'] = vwap_analysis['Volume'].cumsum()
    vwap_analysis['Entry'] = round(vwap_analysis['Cumulative_LTPV'] / vwap_analysis['Cumulative_Volume'], 2)
    vwap_analysis['Exit'] = round(vwap_analysis['Cumulative_HTPV'] / vwap_analysis['Cumulative_Volume'], 2)
    return vwap_analysis[-1:].copy()

def build_analysis_table(summary_df, api_key, margin_of_safety=0.9, vwap_days=126, graham_margin=0.95):
    """
    Builds a portfolio analysis table using financial data from Alpha Vantage API,
    with ticker symbols extracted from the provided summary DataFrame.

    Parameters:
    summary_df (pd.DataFrame): Summary DataFrame with at least a 'TICKER' column.
    api_key (str): Alpha Vantage API key for fetching financial data.
    margin_of_safety (float): Margin of safety for VWAP buy threshold (default: 0.9).
    vwap_days (int): Number of days for VWAP calculation (default: 126).
    graham_margin (float): Margin for Graham buy threshold (default: 0.95).

    Returns:
    list: List of lists containing analysis data for each ticker:
          [symbol, market_price, buy_threshold, graham_buy_threshold, exit_price,
           pe_ratio, pb_ratio, dividend_yield, decision]
    """
    # Extract unique tickers from summary_df
    ticker_symbols = [{'symbol': ticker, 'is_etf': False} for ticker in summary_df['TICKER'].unique()]
    
    portfolio = []
    etf_list = ['FBTC', 'ITA', 'SCHF', 'SCHE','SCHX', 'AOA', 'AOK', 'AOM',
                'AOR', 'BLV', 'SCHA', 'SCHD', 'SCHH', 'SCHM', 'SCHP', 'SCHR',
                'SCHZ']
    
    for ticker in ticker_symbols:
        symbol = ticker['symbol']
        is_etf = symbol in etf_list
        
        # Get price data
        raw_data = get_history(symbol, api_key)
        if raw_data is None:
            portfolio.append([symbol, None, None, None, None, None, None, None, "Error"])
            continue
        
        # Get fundamentals (skip for ETFs)
        current_price = raw_data['Close'].iloc[-1]  # Use Close for fundamental calcs
        fundamentals = None if is_etf else get_fundamentals(symbol, api_key, current_price)
        if not is_etf and fundamentals is None:
            portfolio.append([symbol, None, None, None, None, None, None, None, "Error"])
            continue
        
        # Calculate VWAP
        vwap_data = calculate_vwap(raw_data, days=vwap_days)
        if vwap_data is None:
            portfolio.append([symbol, None, None, None, None, None, None, None, "Error"])
            continue
        
        # Extract data
        market_price = round(raw_data['Close'].iloc[-1], 2)  # Use Close for buys
        entry_price = round(vwap_data['Entry'].iloc[0], 2)
        exit_price = round(vwap_data['Exit'].iloc[0], 2)
        buy_threshold = round(entry_price * margin_of_safety, 2)  # 10% margin

        # Graham buy threshold (for stocks only)
        graham_buy_threshold = None
        if not is_etf:
            if fundamentals['eps'] > 0 and fundamentals['book_value'] > 0:
                # Calculate desired price where P/E × P/B = 38
                desired_price = math.sqrt(38 * fundamentals['eps'] * fundamentals['book_value'])
                graham_buy_threshold = round(desired_price * graham_margin, 2)  # 5% margin
            else:
                graham_buy_threshold = buy_threshold  # Default to VWAP threshold
        
        # Volume filter: 20% of 21-day average
        avg_volume = raw_data['Volume'][-21:].mean()
        today_volume = raw_data['Volume'].iloc[-1]
        volume_ok = today_volume >= avg_volume * 0.2
        
        # Graham's fundamental checks (for stocks only)
        graham_ok = True
        if not is_etf:
            graham_ok = (
                (fundamentals['pe_ratio'] < 19 and fundamentals['pb_ratio'] < 2.0) or
                (fundamentals['pe_ratio'] * fundamentals['pb_ratio'] < 38 and 
                 fundamentals['pe_ratio'] < 100 and fundamentals['pb_ratio'] < 10)
                 ) and fundamentals['dividend_yield'] >= 0 and fundamentals['debt_to_equity'] < 2
        
        # Decision logic
        decision = "Hold"
        if market_price <= min(buy_threshold, graham_buy_threshold or float('inf')) and volume_ok and graham_ok:
            decision = "Buy"
        elif market_price >= exit_price and volume_ok:
            decision = "Sell"
        
        # Prepare fundamentals for output
        pe_ratio = None if is_etf else fundamentals['pe_ratio']
        pb_ratio = None if is_etf else fundamentals['pb_ratio']
        dividend_yield = None if is_etf else fundamentals['dividend_yield']
        
        portfolio.append([
            symbol, market_price, buy_threshold, graham_buy_threshold, exit_price,
            pe_ratio, pb_ratio, dividend_yield, decision
        ])

        # Convert portfolio list to DataFrame with specified column names
        portfolio_df = pd.DataFrame(portfolio, columns=[
            'ticker', 'price', 'entry_low', 'entry_val', 'exit',
            'P/E', 'P/B', 'DivYield', 'rating'
            ])
        
        # Minimal delay for server stability (75 calls/minute = ~0.8 seconds/call)
        time.sleep(0.1)
    
    return portfolio_df

In [46]:
ira_cash = 50093.69
ira_historical_return = 0.9934

brokerage_cash = 6828.54
brokerage_historical_return = 0.4589

york_401k_cash = 2900

In [ ]:
def build_portfolio_df(portfolio_df, portfolio_price_data, 
                       historical_return, inception_date, 
                       desired_total_exposure=0.9, cash_pos=0,
                       run_date=None):
    quant_rankings = list(portfolio_df['QUANT_RATING'])
    quant_rankings.sort(reverse=True)
    quant_threshold = quant_rankings[19]
    portfolio_df_filtered = portfolio_df[(portfolio_df['TOTAL_QUANTITY'] > 0) | (portfolio_df['QUANT_RATING'] > quant_rankings[9])]
    portfolio_df_sorted = portfolio_df_filtered.sort_values(by='QUANT_RATING', ascending=False).reset_index(drop=True)

    if run_date is None:
        run_date = datetime.now().date()
    run_date = pd.to_datetime(run_date)

    # Merge portfolio holdings with price data for each position
    final_portfolio_df = pd.merge(
            portfolio_df_sorted,
            portfolio_price_data, 
            left_on='TICKER',
            right_on='ticker',
            how='left'
        )
    final_portfolio_df.drop(columns=['ticker'], inplace=True) # remove duplicated column

    # Calculate total value of each held position
    final_portfolio_df['VALUE'] = round(final_portfolio_df['TOTAL_QUANTITY'] * final_portfolio_df['price'], ndigits=2)

    # Calculate value of portfolio, portfolio weights, total return and CAGR for each position
    portfolio_total = final_portfolio_df['VALUE'].sum() + cash_pos
    final_portfolio_df['PW%'] = round(final_portfolio_df['VALUE'] / portfolio_total * 100, ndigits=2)
    final_portfolio_df.loc[final_portfolio_df['TOTAL_COST'] < 0, 'AVG_PURCHASE_PRICE'] = 0.01
    final_portfolio_df['TOTAL_RETURN'] = final_portfolio_df['VALUE'] - final_portfolio_df['TOTAL_COST']
    final_portfolio_df['ROI'] = round(final_portfolio_df['TOTAL_RETURN'] / final_portfolio_df['TOTAL_COST'] * 100, ndigits=4)

    # Calculate desired target price to optimize return
    portfolio_days_held = (run_date - inception_date).days
    portfolio_years_held = portfolio_days_held / 360
    portfolio_cagr = (1+historical_return)**(1/portfolio_years_held)-1
    alpha_age = (run_date.date() - pd.to_datetime("07-01-2022", format="%m-%d-%Y").date()).days / 360
    alpha_return = (1+2.0451)**(1/alpha_age)-1
    desired_return = min(portfolio_cagr, alpha_return)
    final_portfolio_df['YEARS_HELD'] = ((run_date - final_portfolio_df['WEIGHTED_AVG_PURCHASE_DATE']).dt.days) / 360
    final_portfolio_df['TARGET'] = round(final_portfolio_df['AVG_PURCHASE_PRICE'] * (1+desired_return)**final_portfolio_df['YEARS_HELD'], ndigits=2)
    final_portfolio_df['CAGR'] = round(((1+final_portfolio_df['ROI']/100) ** (1/final_portfolio_df['YEARS_HELD']) - 1) * 100, ndigits=2)

    # Calculate quant-based portfolio allocation and position adjustments to meet desired allocation
    desired_position_size = portfolio_total * desired_total_exposure * (1/20)
    final_portfolio_df['DESIRED_POS'] = desired_position_size * final_portfolio_df['QUANT_RATING'] / 5
    final_portfolio_df.loc[final_portfolio_df['QUANT_RATING'] < quant_threshold, 'DESIRED_POS'] = 0
    final_portfolio_df['POS_ADJUSTMENT'] = round((final_portfolio_df['DESIRED_POS'] - final_portfolio_df['VALUE']) / final_portfolio_df['price'], ndigits=0)

    # Sorta final portfolio table
    final_portfolio_df = final_portfolio_df.sort_values(by='PW%', ascending=False).reset_index(drop=True)

    return final_portfolio_df

In [48]:
def buys_and_sells_tables(portfolio_df):
    actions_df = portfolio_df[['TICKER', 'TOTAL_QUANTITY', 'POS_ADJUSTMENT', 
                               'price', 'entry_low', 'entry_val', 'exit', 'TARGET', 'rating']].copy()
    actions_df['ACTION'] = abs(actions_df['POS_ADJUSTMENT'] / actions_df['TOTAL_QUANTITY']) > 0.2
    buys_df = actions_df[(actions_df['POS_ADJUSTMENT'] > 0) & (actions_df['ACTION'] == True)]
    sells_df = actions_df[(actions_df['POS_ADJUSTMENT'] < 0) & (actions_df['ACTION'] == True)]
    buys_columns = ['TICKER', 'POS_ADJUSTMENT', 'price', 'entry_low', 'entry_val', 'rating']
    sells_columns = ['TICKER', 'POS_ADJUSTMENT', 'price', 'exit', 'TARGET', 'rating']
    buys_final = buys_df[buys_columns].copy()
    sells_final = sells_df[sells_columns].copy()

    return buys_final, sells_final

In [49]:
ira_price_data = build_analysis_table(ira_df, api_key, margin_of_safety=.975, vwap_days=63)

In [ ]:
ira_portfolio_df = build_portfolio_df(ira_df, ira_price_data, 
                                      historical_return=ira_historical_return,
                                      inception_date=pd.to_datetime("11-15-2023", format="%m-%d-%Y"),
                                      cash_pos=ira_cash)

In [51]:
ira_buys_final, ira_sells_final = buys_and_sells_tables(ira_portfolio_df)

In [80]:
round(ira_portfolio_df['VALUE'].sum() + ira_cash, ndigits=2)

142713.72

In [53]:
ira_buys_final

,TICKER,POS_ADJUSTMENT,price,entry_low,entry_val,rating
5,EZPW,74.0,15.64,13.43,26.60,Sell
6,LRN,7.0,163.53,137.78,83.22,Sell
8,CLS,10.0,182.91,150.45,47.34,Sell
9,UBER,19.0,93.74,85.78,45.92,Sell
12,ALL,13.0,213.63,192.59,252.71,Sell
13,TMUS,10.0,259.01,228.73,140.86,Sell
14,SKYW,24.0,113.44,103.75,142.49,Sell
15,APP,3.0,412.38,359.44,29.30,Sell
17,COMM,208.0,15.71,9.54,2.21,Sell
20,CCL,142.0,29.57,25.68,21.84,Sell


In [54]:
ira_sells_final

,TICKER,POS_ADJUSTMENT,price,exit,TARGET,rating
0,ARQT,-498.0,15.79,14.78,18.24,Sell
3,WLDN,-55.0,103.18,84.50,37.29,Sell
7,QTWO,-61.0,75.44,86.07,80.54,Hold
10,PYPL,-57.0,68.08,72.87,73.86,Hold
11,OKTA,-40.0,91.03,103.12,100.84,Hold
16,WFC,-41.0,78.18,79.17,71.64,Hold
18,LC,-192.0,15.18,13.62,6.91,Sell
19,CAAP,-118.0,20.73,20.63,23.56,Sell


In [55]:
alpha_watch_list = ['MU', 'PGY', 'PSIX']
ira_price_data[ira_price_data['ticker'].isin(alpha_watch_list)]

,ticker,price,entry_low,entry_val,exit,P/E,P/B,DivYield,rating


In [56]:
ira_portfolio_df[['TICKER', 'QUANT_RATING', 'TOTAL_QUANTITY', 
                  'AVG_PURCHASE_PRICE', 'TOTAL_COST',
                  'price', 'VALUE', 'TOTAL_RETURN', 'ROI']]

,TICKER,QUANT_RATING,TOTAL_QUANTITY,AVG_PURCHASE_PRICE,TOTAL_COST,price,VALUE,TOTAL_RETURN,ROI
0,ARQT,3.22,498.0,15.932,7934.2500,15.79,7863.42,-70.8300,-0.8927
1,ATGE,4.61,52.0,116.372,6051.3625,130.46,6783.92,732.5575,12.1057
2,EAT,4.46,39.0,124.038,4837.4853,152.79,5958.81,1121.3247,23.1799
3,WLDN,3.48,55.0,36.080,1984.3920,103.18,5674.90,3690.5080,185.9768
4,DXPE,4.76,49.0,105.590,5173.8900,114.89,5629.61,455.7200,8.8081
5,EZPW,4.75,316.0,15.460,4885.3600,15.64,4942.24,56.8800,1.1643
6,LRN,4.65,30.0,107.868,3236.0438,163.53,4905.90,1669.8562,51.6018
7,QTWO,3.11,61.0,79.148,4828.0223,75.44,4601.84,-226.1823,-4.6848
8,CLS,4.96,25.0,0.010,-86.3962,182.91,4572.75,4659.1462,-5392.7675
9,UBER,4.69,45.0,40.612,1827.5206,93.74,4218.30,2390.7794,130.8209


In [57]:
time.sleep(300)

In [58]:
b_analysis_df = build_analysis_table(brokerage_df, api_key, margin_of_safety=.975, vwap_days=63)

In [59]:
ira_watch_list = ['MU', 'PGY', 'PSIX']
b_analysis_df[b_analysis_df['ticker'].isin(ira_watch_list)]

,ticker,price,entry_low,entry_val,exit,P/E,P/B,DivYield,rating
28,MU,117.21,110.79,88.67,117.44,21.99,2.725,0.0037,Hold
31,PGY,30.29,24.35,24.35,27.42,inf,6.530,0.0000,Sell
34,PSIX,80.73,70.34,30.15,80.66,16.62,14.790,0.0000,Sell


In [ ]:
brokerage_portfolio_df = build_portfolio_df(brokerage_df, b_analysis_df, 
                                            historical_return=brokerage_historical_return,
                                            inception_date=pd.to_datetime("12-26-2023", format="%m-%d-%Y"),
                                            cash_pos=brokerage_cash)

In [79]:
round(brokerage_portfolio_df['VALUE'].sum() + brokerage_cash, ndigits=2)

26072.4

In [62]:
brokerage_portfolio_df = brokerage_portfolio_df[(brokerage_portfolio_df['TOTAL_QUANTITY'] > 0) | (brokerage_portfolio_df['price'] < 100)]

In [63]:
brokerage_buys_final, brokerage_sells_final = buys_and_sells_tables(brokerage_portfolio_df)

In [64]:
brokerage_buys_final

,TICKER,POS_ADJUSTMENT,price,entry_low,entry_val,rating
11,SSRM,18.0,16.45,12.63,21.20,Sell
12,NGD,71.0,5.27,4.48,3.15,Sell
13,SKYW,4.0,113.44,103.75,142.49,Sell
14,UBER,5.0,93.74,85.78,45.92,Sell
15,PGY,18.0,30.29,24.35,24.35,Sell
17,CCL,37.0,29.24,25.75,21.60,Sell


In [65]:
brokerage_sells_final

,TICKER,POS_ADJUSTMENT,price,exit,TARGET,rating
0,EZPW,-69.0,15.64,14.18,16.67,Sell
1,PSIX,-10.0,80.73,80.66,83.04,Sell
2,FBTC,-16.0,98.62,99.18,93.37,Hold
3,OKTA,-17.0,91.03,103.12,103.54,Hold
4,ARQT,-83.0,15.77,14.76,17.56,Sell
5,QTWO,-15.0,75.44,86.07,77.16,Hold
6,FRSH,-85.0,13.28,14.32,13.83,Hold
7,PYPL,-16.0,68.08,72.87,70.73,Hold
8,LX,-161.0,6.39,7.17,6.60,Hold
16,WFC,-8.0,78.18,79.17,76.78,Hold


In [66]:
brokerage_portfolio_df[['TICKER', 'QUANT_RATING', 'TOTAL_QUANTITY', 
                        'AVG_PURCHASE_PRICE', 'TOTAL_COST',
                        'price', 'VALUE', 'TOTAL_RETURN', 'ROI']]

,TICKER,QUANT_RATING,TOTAL_QUANTITY,AVG_PURCHASE_PRICE,TOTAL_COST,price,VALUE,TOTAL_RETURN,ROI
0,EZPW,4.75,140.0,15.773,2208.153384,15.64,2189.60,-18.553384,-0.8402
1,PSIX,4.98,24.0,82.730,1985.520000,80.73,1937.52,-48.000000,-2.4175
2,FBTC,4.46,16.0,86.612,1385.790000,98.62,1577.92,192.130000,13.8643
3,OKTA,3.35,17.0,99.535,1692.100000,91.03,1547.51,-144.590000,-8.5450
4,ARQT,3.22,83.0,15.996,1327.643700,15.77,1308.91,-18.733700,-1.4110
5,QTWO,3.11,15.0,76.435,1146.530500,75.44,1131.60,-14.930500,-1.3022
6,FRSH,3.12,85.0,13.667,1161.666300,13.28,1128.80,-32.866300,-2.8292
7,PYPL,3.42,16.0,68.292,1092.669800,68.08,1089.28,-3.389800,-0.3102
8,LX,3.35,161.0,6.534,1052.029000,6.39,1028.79,-23.239000,-2.2090
9,COMM,4.99,63.0,0.010,-463.431000,15.71,989.73,1453.161000,-313.5658


In [67]:
york_401k_sorted = york_401k_list.sort_values(by='QUANT_RATING', ascending=False).reset_index(drop=True)
york_401k_preferred = york_401k_sorted[:5]

fouroh1k_price_data = build_analysis_table(york_401k_preferred, api_key, margin_of_safety=.975, vwap_days=63)

In [68]:
fouroh1k_price_data

,ticker,price,entry_low,entry_val,exit,P/E,P/B,DivYield,rating
0,ITA,194.27,181.30,None,188.76,None,None,None,Sell
1,SCHF,22.82,21.45,None,22.15,None,None,None,Sell
2,SCHE,31.40,29.46,None,30.39,None,None,None,Sell
3,SCHX,25.24,23.62,None,24.47,None,None,None,Sell
4,AOA,85.13,80.17,None,82.78,None,None,None,Sell


In [69]:
'''def setup_driver():
    chrome_options = Options()
    chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")  # Connect to existing Chrome
    driver = webdriver.Chrome(options=chrome_options)  # No need for user-data-dir here
    return driver

def scrape_quant_ratings(symbol, driver, days=63):
    url = f'https://seekingalpha.com/symbol/{symbol}/ratings/quant-ratings'
    driver.get(url)
    time.sleep(5)  # Extra delay for dynamic content
    
    try:
        # Wait for table to load
        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.TAG_NAME, 'table')))
    except Exception as e:
        driver.save_screenshot(f'error_{symbol}.png')
        with open(f'page_source_{symbol}.html', 'w', encoding='utf-8') as f:
            f.write(driver.page_source)
        print(f"Page load failed for {symbol}: {e}. Saved screenshot and page source for debugging.")
        raise Exception(f"Failed to load page for {symbol}")
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Find the quant rating history table
    table = soup.find('table')
    
    if not table:
        raise Exception("Table not found")
    
    rows = table.find_all('tr')[1:]  # Skip header
    data = []
    for row in rows:
        cols = row.find_all('td')
        if len(cols) >= 4:
            date_str = cols[0].text.strip()
            try:
                date = datetime.strptime(date_str, '%m/%d/%Y')
            except ValueError:
                continue
            quant_score_str = cols[3].text.strip()
            try:
                quant_score = float(quant_score_str)
            except ValueError:
                quant_score = np.nan
            data.append({'date': date, 'quant_score': quant_score})
    
    df = pd.DataFrame(data)
    df = df.sort_values('date', ascending=False)
    
    # Filter last 63 days (current date as 02:24 PM MDT, Aug 10, 2025)
    end_date = datetime(2025, 8, 10, 14, 24)  # 2:24 PM MDT
    start_date = end_date - timedelta(days=days)
    df_filtered = df[(df['date'] >= start_date) & (df['date'] <= end_date)]
    
    return df_filtered

def compute_stats(df):
    scores = df['quant_score'].dropna()
    if len(scores) == 0:
        return {'average': np.nan, 'median': np.nan}
    return {
        'average': scores.mean(),
        'median': scores.median()
    }

# Main function
def main(companies):
    driver = setup_driver()
    try:
        results = {}
        for symbol in companies:
            try:
                df = scrape_quant_ratings(symbol, driver)
                stats = compute_stats(df)
                results[symbol] = stats
            except Exception as e:
                print(f"Error for {symbol}: {e}")
                results[symbol] = {'average': np.nan, 'median': np.nan}
        
        # Output results
        results_df = pd.DataFrame.from_dict(results, orient='index')
        print(results_df)
    finally:
        driver.quit()

# Example usage
companies = ['DXPE', 'AAPL', 'GOOG']  # Replace with your list
main(companies)
'''

'def setup_driver():\n    chrome_options = Options()\n    chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")  # Connect to existing Chrome\n    driver = webdriver.Chrome(options=chrome_options)  # No need for user-data-dir here\n    return driver\n\ndef scrape_quant_ratings(symbol, driver, days=63):\n    url = f\'https://seekingalpha.com/symbol/{symbol}/ratings/quant-ratings\'\n    driver.get(url)\n    time.sleep(5)  # Extra delay for dynamic content\n    \n    try:\n        # Wait for table to load\n        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.TAG_NAME, \'table\')))\n    except Exception as e:\n        driver.save_screenshot(f\'error_{symbol}.png\')\n        with open(f\'page_source_{symbol}.html\', \'w\', encoding=\'utf-8\') as f:\n            f.write(driver.page_source)\n        print(f"Page load failed for {symbol}: {e}. Saved screenshot and page source for debugging.")\n        raise Exception(f"Failed to load page for {symbol

In [70]:
# Machine Learning

In [71]:
# Data processing and clearning
# Must be in numpy array or tf.Dataset object format

In [72]:
# Feature selection and normalization

In [73]:
# Build model

In [74]:
# Train model

In [75]:
# Evaluate model

In [76]:
# Refine model through hyperparameter tuning